In [1]:
import pickle
from attention import *
from torch.utils.data import DataLoader
from pytorch_util import RAdam,trainable_parameter
from torch.optim.lr_scheduler import ReduceLROnPlateau
#from pytorch_transformers import AdamW,WarmupLinearSchedule

In [2]:
batch_size = 1024
dim = 512
head_d = 8
head = SimplyInteraction #SimplyHead
encoder_layer,decoder_layer = 4,2
epochs = 10
clip = 0.1
logLoss = True
EncoderLayer=TransformerEncoderLayer
DecoderLayer=TransformerDecoderLayer

dim_feedforward = 1024
dropout = 0.05
lr = 6e-4

In [3]:
with open('../Data/train_data_attention_node.pickle', 'rb') as handle:
    train_node = pickle.load(handle)
with open('../Data/train_data_attention_edge.pickle', 'rb') as handle:
    train_edge = pickle.load(handle)
with open('../Data/train_data_attention_edge_y.pickle', 'rb') as handle:
    train_y = pickle.load(handle)
with open('../Data/train_data_ind.pickle', 'rb') as handle:
    ind = pickle.load(handle)

In [4]:
train_dl = attentionDataset(train_node[:4000000],train_edge[:4000000],ind[:4000000],train_y[:4000000])
train_dl = DataLoader(train_dl,shuffle=True,batch_size=batch_size,collate_fn=collate_fn2,num_workers=4)
val_dl = attentionDataset(train_node[4000000:],train_edge[4000000:],ind[4000000:],train_y[4000000:])
val_dl = DataLoader(val_dl,shuffle=True,batch_size=batch_size,collate_fn=collate_fn2,num_workers=4)

In [10]:
model=Attention3(dim,encoder_layer,head_d,head,EncoderLayer,
                 dropout=dropout,dim_feedforward=dim_feedforward).to('cuda')
paras = trainable_parameter(model)
opt = RAdam(paras,lr=lr,weight_decay=1e-2)
scheduler = ReduceLROnPlateau(opt, 'min',factor=0.5,patience=5,min_lr=1e-04)
model, opt = amp.initialize(model, opt, opt_level="O1")

Selected optimization level O2:  FP16 training with FP32 batchnorm and FP32 master weights.

Defaults for this optimization level are:
enabled                : True
opt_level              : O2
cast_model_type        : torch.float16
patch_torch_functions  : False
keep_batchnorm_fp32    : True
master_weights         : True
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O2
cast_model_type        : torch.float16
patch_torch_functions  : False
keep_batchnorm_fp32    : True
master_weights         : True
loss_scale             : dynamic


In [6]:
# out,mask,edge,y = next(iter(train_dl))
# out,mask,edge,y = out.to('cuda:0'),mask.to('cuda:0'),edge.to('cuda:0'),y.to('cuda:0')

# opt.zero_grad()
# loss,loss_perType = model(out,mask,edge,y,logLoss)
# loss.backward()

# [(n,p.std()) for n,p in model.named_parameters()]

# [(n,p.grad.mean()) for n,p in model.named_parameters()]

In [10]:
model,bestOpt,bestWeight = train_type2(opt,model,epochs,train_dl,val_dl,paras,clip,\
                                       scheduler=scheduler,logLoss=logLoss)

epoch:0, train_loss: +0.407, val_loss: +0.238, 
train_vector: +1.36|+1.86|+0.23|+0.32|+0.04|+0.15|+0.01|-0.72, 
val_vector  : +1.31|+1.53|-0.09|+0.07|+0.29|-0.01|-0.36|-0.84

epoch:1, train_loss: +0.010, val_loss: -0.071, 
train_vector: +0.99|+1.21|-0.08|-0.08|-0.44|-0.13|-0.35|-1.05, 
val_vector  : +1.03|+1.10|-0.20|-0.30|-0.04|-0.30|-0.78|-1.07

epoch:2, train_loss: -0.197, val_loss: -0.194, 
train_vector: +0.86|+0.75|-0.25|-0.27|-0.64|-0.28|-0.53|-1.21, 
val_vector  : +0.88|+1.11|-0.39|-0.37|-0.32|-0.51|-0.78|-1.16

epoch:3, train_loss: -0.324, val_loss: -0.250, 
train_vector: +0.75|+0.56|-0.37|-0.39|-0.77|-0.40|-0.65|-1.32, 
val_vector  : +1.01|+0.79|-0.45|-0.36|-0.43|-0.57|-0.84|-1.14

epoch:4, train_loss: -0.393, val_loss: -0.209, 
train_vector: +0.70|+0.48|-0.44|-0.46|-0.84|-0.46|-0.72|-1.39, 
val_vector  : +1.13|+1.20|-0.53|-0.37|-0.45|-0.58|-0.88|-1.18

epoch:5, train_loss: -0.450, val_loss: -0.577, 
train_vector: +0.65|+0.44|-0.50|-0.52|-0.91|-0.52|-0.78|-1.45, 
val_vector  :

In [8]:
# O1
model,bestOpt,bestWeight = train_type2(opt,model,epochs,train_dl,val_dl,paras,clip,\
                                       scheduler=scheduler,logLoss=logLoss)

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 65536.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
epoch:0, train_loss: +0.392, val_loss: +0.190, 
train_vector: +1.37|+1.76|+0.18|+0.34|+0.02|+0.16|+0.00|-0.70, 
val_vector  : +0.95|+1.71|-0.16|+0.18|+0.22|-0.12|-0.44|-0.82

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
epoch:1, train_loss: -0.018, val_loss: -0.034, 
train_vector: +0.98|+1.05|-0.14|-0.04|-0.46|-0.11|-0.40|-1.04, 
val_vector  : +1.02|+1.24|-0.28|-0.15|-0.40|-0.20|-0.57|-0.93

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
epoch:2, train_loss: -0.190, val_loss: -0.176, 
train_vector: +0.83|+0.79|-0.27|-0.25|-0.65|